In [9]:
from epymetheus import Trade, TradeHistory, TradeStrategy, Universe, Transaction, Backtester
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import datetime
import pathlib
from dateutil.relativedelta import relativedelta
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

In [10]:
class SimpleMeanReversion(TradeStrategy):
    """
    A simple Mean Reversion strategy that buys stocks for a month with
    the lowest percentile one month returns and sells the highest percentile returns.

    Parameters
    ----------
    - percentile : float
        The threshold to buy or sell.
        E.g. If 0.1, buy/sell stocks with returns of lowest/highest 10%.
    """

    def logic(self, universe, percentile):

        train_period = relativedelta(months=1)
        trade_period = relativedelta(months=1)

        def trade_bds(universe, train_period, trade_period):
            """Yield begin dates of trade periods."""
            d = universe.begin_date + train_period
            while d + trade_period <= universe.end_date:
                yield d
                d += trade_period

        def tot_return(asset, begin_date, end_date):
            """Return total return of asset from begin_date to end_date."""
            b, e = universe.data.at[begin_date, asset], universe.data.at[end_date, asset]
            return e / b - 1

        num_buysell = int(len(universe.data.columns) * percentile)

        for trade_bd in trade_bds(universe, train_period, trade_period):
            train_bd = trade_bd - train_period
            trade_ed = trade_bd + trade_period - relativedelta(days=1)
            train_ed = train_bd + train_period - relativedelta(days=1)

            key = lambda asset: tot_return(asset, train_bd, train_ed)
            buy = sorted(universe.data.columns, key=key)[:num_buysell]
            sell = sorted(universe.data.columns, key=key)[-num_buysell:]
            
            for asset in buy:
                yield Trade(asset=asset, lot=1, begin_date=trade_bd, end_date=trade_ed)
            for asset in sell:
                yield Trade(asset=asset, lot=-1, begin_date=trade_bd, end_date=trade_ed)

In [11]:
csvs = pathlib.Path('../data/JP Equity').glob('*.csv')

universe = Universe.read_csvs(name='JP Equity',
                              csvs=csvs, index_col=0, parse_dates=True,
                              begin_date=datetime.date(2000, 1, 1), 
                              end_date=datetime.date(2000, 12, 31))
# print(universe.name)
# universe.data.head()

In [12]:
strategy = SimpleMeanReversion(percentile=0.1)

In [13]:
backtester = Backtester()

backtester.run(strategy, universe, verbose=True)

Generating trades ...
Evaluating wealth ...
Done.
Runtime : 0.3sec


In [14]:
backtester.wealth_.data

2000-01-01       0.0
2000-01-02       0.0
2000-01-03       0.0
2000-01-04       0.0
2000-01-05       0.0
               ...  
2000-12-27   -1398.0
2000-12-28   -1398.0
2000-12-29   -1398.0
2000-12-30   -1398.0
2000-12-31   -1398.0
Length: 366, dtype: float64